In [ ]:
#TODO: 1. optimize parameters for YOLO nano/small/medium model

# install ultralytics, import yolo
!pip install ultralytics
from ultralytics import YOLO
import cv2
import numpy as np
from google.colab import files

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
# upload data from https://www.pexels.com/video/traffic-flow-in-the-highway-2103099/
data = files.upload()
print("Data are uploaded")

Saving 2103099-hd_1920_1080_30fps.mp4 to 2103099-hd_1920_1080_30fps.mp4
Data are uploaded


In [ ]:
# import yolo
model = YOLO('yolov8n.pt') # another one, if is needed
# TODO: Use yolo models on video: 1. Detect cars, if works -> 2. detect SPZ (car numbers)
# Make new video with used model for detection

100%|██████████| 6.25M/6.25M [00:00<00:00, 111MB/s]


In [ ]:
from ultralytics import YOLO
import cv2
from google.colab import files

# Upload video file
print("Please upload your traffic video file...")

video_path = '/content/2103099-hd_1920_1080_30fps.mp4'
print(f"✅ Uploaded: {video_path}")

# Load YOLO model (using nano version for faster processing)
car_model = YOLO('yolov8n.pt')  # or 'yolov8s.pt' for better balance of speed/accuracy

output_video_path = "car_detection_output.mp4"

# Load video
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Prepare video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Detection data storage
detection_data = []

# Car detection parameters
conf_threshold = 0.3  # Confidence threshold for car detection
iou_threshold = 0.5   # IOU threshold for NMS
box_thickness = 2      # Bounding box thickness
box_color = (0, 255, 0)  # Green color for car boxes

while cap.isOpened(): # read frames
    ret, frame = cap.read()
    if not ret:
        break

    # Detect cars
    results = car_model(frame, conf=conf_threshold, iou=iou_threshold)

    frame_data = {
        "frame_number": int(cap.get(cv2.CAP_PROP_POS_FRAMES)),
        "cars": []
    }

    # Process detections
    for result in results:
        for box in result.boxes:
            if int(box.cls) == 2:  # Class 2 is for cars
                bbox = box.xyxy[0].tolist()
                confidence = float(box.conf)

                # Store car data
                frame_data["cars"].append({
                    "bbox": bbox,
                    "confidence": confidence
                })

                # Draw bounding box
                x1, y1, x2, y2 = map(int, bbox)
                cv2.rectangle(frame,
                              (x1, y1),
                              (x2, y2),
                              box_color,
                              box_thickness)

                # Add confidence text
                cv2.putText(frame,
                           f"Car: {confidence:.2f}",
                           (x1, y1 - 10),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.5,
                           box_color,
                           1)

    detection_data.append(frame_data)
    out.write(frame)

cap.release()
out.release()

print(f"✅ Car detection complete! Output saved as {output_video_path}")
print(f"✅ Detected {sum(len(frame['cars']) for frame in detection_data)} cars total")
print(f"✅ Detection data contains {len(detection_data)} frames")

# For future license plate detection:
print("\nNext steps:")
print("1. Get a license plate detection model")
print("2. Add the license plate detection code where indicated")
print("3. The current data structure is already prepared for LP detection results")

Streaming output truncated to the last 5000 lines.

0: 384x640 16 cars, 1 bus, 1 truck, 8.4ms
Speed: 3.4ms preprocess, 8.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 cars, 1 bus, 1 truck, 9.4ms
Speed: 4.9ms preprocess, 9.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 1 bus, 1 truck, 11.4ms
Speed: 3.9ms preprocess, 11.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 1 bus, 1 truck, 10.7ms
Speed: 3.7ms preprocess, 10.7ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 cars, 1 bus, 1 truck, 7.9ms
Speed: 3.4ms preprocess, 7.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 cars, 1 bus, 1 truck, 8.4ms
Speed: 3.4ms preprocess, 8.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 cars, 1 truck, 10.3ms
Speed: 3.2ms preprocess, 10.3ms inference, 1.9ms postprocess per image at 

In [ ]:
files.download(output_video_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>